In [1]:
from d4pg_agent import Agent
from d4pg_model import D4PGNet as Net
from replay_buffer import AgentMemory
from d4pg_utils import hard_update, soft_update
import torch
import torch.optim as optim

from collections import deque
import numpy as np

from config import *

In [2]:
agent = Agent()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
local_net = Net(agent.state_size, agent.action_size)
target_net = Net(agent.state_size, agent.action_size)

hard_update(local_net, target_net)

actor_opt = optim.Adam(local_net.actor.parameters(), lr=LR)
critic_opt = optim.Adam(local_net.critic.parameters(), lr=LR)

agent_memory = AgentMemory(batch_size=BATCH_SIZE, buffer_size=MEMORY_BUFFER)

In [5]:
episodes = int(1e4)

In [ ]:
scores = []
scores_window = deque(maxlen=100)

for e in range(1, episodes+1):

    agent.reset()

    for i in range(agent.max_steps):
        agent.step(local_net, agent_memory)

        if agent_memory.has_enough_memory() and (agent.step_count % LEARN_EVERY == 0):
            trajectories = agent_memory.sample()
            agent.learn(actor_opt, critic_opt, local_net, target_net, trajectories)
            soft_update(local_net, target_net, TAU)

    scores.append(agent.scores.mean())
    scores_window.append(agent.scores.mean())

    if e % 10 == 0:
        print('Episode %d, avg score: %.5f' % (e, np.mean(scores_window)))

    if np.mean(scores_window) >= 30:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e - 100,
                                                                                     np.mean(scores_window)))
        torch.save(local_net.actor.state_dict(), 'checkpoints/actor_checkpoint_%d.pth' % e)
        torch.save(local_net.critic.state_dict(), 'checkpoints/critic_checkpoint_%d.pth' % e)

Episode 10, avg score: 0.60570
Episode 20, avg score: 0.61030
Episode 30, avg score: 0.62960
Episode 40, avg score: 0.63644
Episode 50, avg score: 0.64655
Episode 60, avg score: 0.64367
Episode 70, avg score: 0.64484
Episode 80, avg score: 0.64533
Episode 90, avg score: 0.64588
Episode 100, avg score: 0.64515
Episode 110, avg score: 0.65356
Episode 120, avg score: 0.65568
Episode 130, avg score: 0.65283
Episode 140, avg score: 0.64782
Episode 150, avg score: 0.64145
Episode 160, avg score: 0.64621
Episode 170, avg score: 0.64907
Episode 180, avg score: 0.64730
Episode 190, avg score: 0.64612
Episode 200, avg score: 0.64340
Episode 210, avg score: 0.64008
Episode 220, avg score: 0.63897
Episode 230, avg score: 0.63843
Episode 240, avg score: 0.64243
Episode 250, avg score: 0.64520
Episode 260, avg score: 0.64200
Episode 270, avg score: 0.63597
Episode 280, avg score: 0.63784
Episode 290, avg score: 0.64283
Episode 300, avg score: 0.64936
Episode 310, avg score: 0.64704
Episode 320, avg 